In [2]:
from crewai import Crew, Process, Agent, Task
from langchain.prompts import PromptTemplate
from crewai_tools import BaseTool
import nest_asyncio
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from langchain_groq import ChatGroq
from llama_index.llms.openai import OpenAI

#Settings.llms=ChatGroq(model="llama3-8b-8192")
Settings.llm = OpenAI(model="gpt-3.5-turbo")
# embedding model
Settings.embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader

nest_asyncio.apply()

c:\Users\sansi\OneDrive\Desktop\devfolio\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from crewai import Agent
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")


PDF_Search_Agent=Agent(
        role='Tamil Nadu Government Help-Desk',
    goal='Extract concise and relevant information from the PDF based on the user query.',
    backstory='An Assistant with years of experience in Tamil Nadu Goventment Schemes in the domains of social service, healthcare and legal services.',
     )

Web_Search_Agent=Agent(
        role='Comprehensive Services Information Specialist',
        goal='Provide detailed and up-to-date information on social services, healthcare services, and legal services in Tamil Nadu.',
        backstory='A seasoned expert in navigating complex service landscapes in Tamil Nadu, adept at synthesizing information from diverse sources. This specialist combines deep knowledge of local public service systems with advanced research capabilities to deliver clear, actionable insights on available services, ensuring citizens in Tamil Nadu are well-informed and able to access essential resources effectively.',
        
        )

Master_Agent = Agent(
    role='Master Agent',
    goal='Check the output of the other tasks and prioritize which answer is the best suitable for the query asked by the user, adding details and citations as needed.',
    backstory='master_agent is a proficient legal detailed context summarizer with the ability to synthesize information from various sources to deliver the most accurate and comprehensive response.',
 
)


In [4]:

class PDF_search(BaseTool):
    name: str = "PDF reader"
    description: str = "Access the documents and answer the query, also give the corresponding document name."

    def _run(self, user_query: str) -> str:

        

        if 'index_store' not in os.listdir():

            documents = SimpleDirectoryReader("./data", recursive = True).load_data()

            # chunk_size of 1024 is a good default value
            splitter = SentenceSplitter(chunk_size=1024)
            # Create nodes from documents
            nodes = splitter.get_nodes_from_documents(documents)
            
            # vector store index
            vector_index = VectorStoreIndex(nodes)

            os.makedirs("./index_store/vector", exist_ok=True)


            vector_index.storage_context.persist(persist_dir="./index_store/vector")

        else:
            # rebuild storage context
            vector_storage_context = StorageContext.from_defaults(persist_dir="./index_store/vector")

            # load index
            vector_index = load_index_from_storage(vector_storage_context)


        vector_query_engine = vector_index.as_query_engine()
        response = vector_query_engine.query(f"{user_query}")
        return response


In [5]:
from crewai_tools import SerperDevTool

# Initialize the tool for internet searching capabilities
web_tool = SerperDevTool()
pdf_tool = PDF_search()


In [6]:
Web_Search_Task = Task(
    expected_output="A brief answer to the query asked by  the user ehich helps them understand what the goverment schemes might be available for them.",
    description=""" Display only the linksof top 3 searches
        The agent should gather and provide detailed information on social services, healthcare services, and legal services(according to the context) available in Tamil Nadu. This includes finding and summarizing relevant policies, eligibility criteria, application procedures, and benefits for each type of service. The goal is to combine this information into a comprehensive summary that covers all three categories, ensuring that the output provides a clear and detailed overview of the services available in Tamil Nadu in short and also privide the official links also."""
    ,agent = Web_Search_Agent,
    tools = [web_tool],
)
PDF_search_task = Task(
    expected_output="A clear and concise response to the user's query based on the relevant information extracted from the PDF document in short.",
    description="""
        The agent should analyze the provided PDF document to extract pertinent information related to the user's query. This includes identifying and summarizing key details that directly address the query. The goal is to return a well-organized and accurate response that effectively answers the user's question based on the content of the PDF.
    """,
    agent = PDF_Search_Agent,
    tools = [pdf_tool],
)
master_task= Task(
    description = 'summarize the content given to you ',
    expected_output='An answer to the question in 2 to 3 lines and be in detail.',
    agent = Master_Agent,
    context=[Web_Search_Task ,PDF_search_task])


In [7]:
crew = Crew(
  agents=[Master_Agent],
  tasks=[PDF_search_task, Web_Search_Task,master_task],
  verbose=1,
  full_output = True,
  process=Process.sequential,
  output_log_file=True,
)

query = input("Enter your question: ")
result = crew.kickoff(inputs = {'query': query})


 [2024-08-01 21:37:59][DEBUG]: == Working Agent: Tamil Nadu Government Help-Desk
 

Tamil Nadu Government provides various healthcare schemes to its residents, including the Chief Minister's Comprehensive Health Insurance Scheme (CMCHIS) which offers coverage for medical expenses, the Amma Master Health Check-up scheme for early detection of diseases, and the Amma Baby Care Kit scheme for newborn care. Additionally, the state government runs initiatives like the Kalaignar Insurance Scheme for Life Saving Treatments and the Thalikku Thangam Thittam for maternity assistance.

 [2024-08-01 21:38:15][DEBUG]: == [Tamil Nadu Government Help-Desk] Task output: Tamil Nadu Government provides various healthcare schemes to its residents, including the Chief Minister's Comprehensive Health Insurance Scheme (CMCHIS) which offers coverage for medical expenses, the Amma Master Health Check-up scheme for early detection of diseases, and the Amma Baby Care Kit scheme for newborn care. Additionally, th

In [8]:
print(result)

### Social Services in Tamil Nadu

1. **Moovalur Ramamirtham Ammaiyar Ninaivu Marriage Assistance Scheme**
   - Provides financial assistance to women from poor families for their marriage.
   - [More Information](https://www.tn.gov.in/scheme)

2. **E.V.R. Maniammaiyar Ninaivu Widow Remarriage Assistance Scheme**
   - Offers financial aid to widows who wish to remarry.
   - [More Information](https://www.tn.gov.in/scheme)

3. **Annai Theresa Ninaivu Orphan Girl Marriage Assistance Scheme**
   - Financial assistance for the marriage of orphan girls.
   - [More Information](https://www.tn.gov.in/scheme)

4. **Free Bus Travel for Women**
   - Free bus travel for women in state-run buses.
   - [More Information](https://www.tn.gov.in/scheme)

5. **Girl Child Protection Scheme**
   - Financial support for the upbringing of girl children.
   - [More Information](https://www.tn.gov.in/scheme)

### Healthcare Services in Tamil Nadu

1. **Chief Minister's Comprehensive Health Insurance Scheme (